In [2]:
# WGAN implementation credit to: https://github.com/eriklindernoren/Keras-GAN/blob/master/wgan/wgan.py

from keras.layers import Input, Dense, Dropout, LeakyReLU, Activation, Lambda, BatchNormalization
from keras.models import Model, load_model
from keras.optimizers import RMSprop
import keras.backend as K
from keras.callbacks import TensorBoard
import numpy as np
from data_pipeline import load_data, save_synthetic
from utils import *
import datetime
import tensorflow as tf
import warnings
from layers import GeneWiseNoise, MinibatchDiscrimination

warnings.filterwarnings('ignore', message='Discrepancy between')

In [3]:
LATENT_DIM = 512
DEFAULT_NOISE_RATE = 0
CHECKPOINTS_DIR = '../checkpoints'

In [4]:
# TODO: Pass function of evaluation scores in train instead of gene symbols

class GAN:
    def __init__(self, data, gene_symbols, latent_dim=LATENT_DIM, noise_rate=DEFAULT_NOISE_RATE,
                 discriminate_batch=False,
                 max_replay_len=None):
        """
        Initialize GAN
        :param data: expression matrix. Shape=(nb_samples, nb_genes)
        :param gene_symbols: list of gene symbols. Shape=(nb_genes,)
        :param latent_dim: number input noise units for the generator
        :param noise_rate: rate of noise being added in the gene-wise layer of the generator (psi/nb_genes).
        :param discriminate_batch: whether to discriminate a whole batch of samples rather than discriminating samples
               individually. NOTE: Not implemented
        :param max_replay_len: size of the replay buffer
        """
        self._latent_dim = latent_dim
        self._noise_rate = noise_rate
        self._data = data
        self._gene_symbols = gene_symbols
        self._nb_samples, self._nb_genes = data.shape
        self._discriminate_batch = discriminate_batch
        self._max_replay_len = max_replay_len

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the discriminator
        self.discriminator = self._build_discriminator()
        if self._discriminate_batch:
            # loss = self._minibatch_binary_crossentropy
            raise NotImplementedError

        self.discriminator.compile(loss=self.wasserstein_loss,
                                   optimizer=optimizer)
        self._gradients_discr = self._gradients_norm(self.discriminator)

        # Build the generator
        self.generator = self._build_generator()
        z = Input(shape=(self._latent_dim,))
        gen_out = self.generator(z)

        # Build the combined model
        self.discriminator.trainable = False
        valid = self.discriminator(gen_out)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
                              optimizer=optimizer)
        self._gradients_gen = self._gradients_norm(self.combined)

        # Prepare TensorBoard callbacks
        time_now = str(datetime.datetime.now())
        log_path = '../logs/discr/{}'.format(time_now)
        self._callback_discr = TensorBoard(log_path)
        self._callback_discr.set_model(self.discriminator)
        log_path = '../logs/gen/{}'.format(time_now)
        self._callback_gen = TensorBoard(log_path)
        self._callback_gen.set_model(self.generator)

        # Initialize replay buffer
        noise = np.random.normal(0, 1, (32, self._latent_dim))
        x_fake = self.generator.predict(noise)
        self._replay_buffer = np.array(x_fake)

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def _build_generator(self):
        """
        Build the generator
        """
        noise = Input(shape=(self._latent_dim,))
        h = noise
        # h = Dropout(0.5)(h)
        
        h = Dense(256)(h)
        h = LeakyReLU(0.2)(h)
        h = Dropout(0.5)(h)
        
        h = Dense(128)(h)
        h = LeakyReLU(0.2)(h)
        h = Dropout(0.5)(h)
        
        h = Dense(256)(h)
        h = LeakyReLU(0.2)(h)
        h = Dropout(0.5)(h)
        
        h = Dense(512)(h)
        h = LeakyReLU(0.2)(h)
        h = Dropout(0.5)(h)
        
        h = Dense(self._nb_genes)(h)
        # h = Activation('tanh')(h)
        # h = LeakyReLU(0.3)(h)
        self._noise_layer = GeneWiseNoise(self._noise_rate)
        h = self._noise_layer(h)
        # h = Activation('tanh')(h)
        model = Model(inputs=noise, outputs=h)
        model.summary()
        return model

    def _build_discriminator(self):
        """
        Build the discriminator
        """
        expressions_input = Input(shape=(self._nb_genes,))
        h = expressions_input
        h = Dropout(0.1)(h)
        h = MinibatchDiscrimination()(h)
        
        h = Dense(512)(h)
        h = LeakyReLU(0.2)(h)
        h = Dropout(0.5)(h)
        
        h = Dense(256)(h)
        h = LeakyReLU(0.2)(h)
        h = Dropout(0.5)(h)
        
        h = Dense(128)(h)
        h = LeakyReLU(0.2)(h)
        h = Dropout(0.5)(h)
        
        h = Dense(1)(h)
        model = Model(inputs=expressions_input, outputs=h)
        model.summary()
        return model

    @staticmethod
    def _write_log(callback, names, logs, epoch):
        """
        Write log to TensorBoard callback
        :param callback: TensorBoard callback
        :param names: list of names for each log. Shape=(nb_logs,)
        :param logs: list of scalars. Shape=(nb_logs,)
        :param epoch: epoch number
        """
        for name, value in zip(names, logs):
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value
            summary_value.tag = name
            callback.writer.add_summary(summary, epoch)
            callback.writer.flush()

    @staticmethod
    def _gradients_norm(model):
        """
        Create Keras function to compute Euclidean norm of gradient vector
        :param model: Keras model for which the norm will be computed
        :return: Keras function to compute Euclidean norm of gradient vector
        """
        grads = K.gradients(model.total_loss, model.trainable_weights)
        summed_squares = tf.stack([K.sum(K.square(g)) for g in grads])
        norm = K.sqrt(K.sum(summed_squares))

        input_tensors = [model.inputs[0],  # input data
                         model.sample_weights[0],  # how much to weight each sample by
                         model.targets[0],  # labels
                         K.learning_phase(),  # train or test mode
                         ]

        return K.function(inputs=input_tensors, outputs=[norm])

    def train(self, epochs, file_name=None, batch_size=32):
        """
        Trains the GAN
        :param epochs: Number of epochs
        :param file_name: Name of the .h5 file in checkpoints. If None, the model won't be saved
        :param batch_size: Batch size
        """
        # Best score for correlation of distance gene matrices
        best_gdxdz = 0

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        for epoch in range(epochs):

            for _ in range(self.n_critic):
                # ----------------------
                #  Train Discriminator
                # ----------------------

                # Select random samples
                idxs = np.random.randint(0, self._nb_samples, batch_size)
                x_real = self._data[idxs, :]

                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self._latent_dim))
                x_fake = self.generate_batch(batch_size)  # self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(x_real, valid)
                d_loss_fake = self.discriminator.train_on_batch(x_fake, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # Clip critic weights
                for l in self.discriminator.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)

            # ----------------------
            #  Train Generator
            # ----------------------

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid)

            # Add generator loss to TensorBoard
            self._write_log(self._callback_gen, ['train_loss'], [g_loss], epoch)

            # ----------------------
            #  Report gradient norms
            # ----------------------

            # Gradient norm TensorBoard log
            if (epoch + 1) % 100 == 0:
                # Get discriminator gradients
                grad_inputs = [noise,  # data
                               np.ones(shape=(batch_size,)),  # Sample weights
                               valid,  # labels
                               0  # set learning phase in TEST mode
                               ]
                g_gen = self._gradients_gen(grad_inputs)[0]

                # Get discriminator gradients
                grad_inputs = [x_real,  # data
                               np.ones(shape=(batch_size,)),  # Sample weights
                               valid,  # labels
                               0  # set learning phase in TEST mode
                               ]
                g_discr_real = self._gradients_discr(grad_inputs)[0]
                grad_inputs = [x_fake,  # data
                               np.ones(shape=(batch_size,)),  # Sample weights
                               fake,  # labels
                               0  # set learning phase in TEST mode
                               ]
                g_discr_fake = self._gradients_discr(grad_inputs)[0]
                g_discr = (g_discr_real + g_discr_fake) / 2

                # Add information to TensorBoard log
                self._write_log(self._callback_discr, ['gradient_norm'], [g_discr], epoch)
                self._write_log(self._callback_gen, ['gradient_norm'], [g_gen], epoch)

                
            # ----------------------
            #  Plot the progress
            # ----------------------
            print('{} [D loss: {:.4f}] [G loss: {:.4f}]'.format(epoch, d_loss, g_loss))

      

    def generate_batch(self, batch_size=32):
        """
        Generate a batch of samples using the generator
        :param batch_size: Batch size
        :return: Artificial samples generated by the generator. Shape=(batch_size, nb_genes)
        """
        noise = np.random.normal(0, 1, (batch_size, self._latent_dim))
        pred = self.generator.predict(noise)

        return pred

    def discriminate(self, expr):
        """
        Discriminates a batch of samples
        :param expr: expressions matrix. Shape=(nb_samples, nb_genes)
        :return: for each sample, probability that it comes from the real distribution
        """
        return self.discriminator.predict(expr)

    def save_model(self, name):
        """
        Saves model to CHECKPOINTS_DIR
        :param name: model id
        """
        self.discriminator.trainable = True
        self.discriminator.save('{}/discr/{}.h5'.format(CHECKPOINTS_DIR, name))
        self.discriminator.trainable = False
        for layer in self.discriminator.layers:  # https://github.com/keras-team/keras/issues/9589
            layer.trainable = False
        self.generator.save('{}/gen/{}.h5'.format(CHECKPOINTS_DIR, name), include_optimizer=False)
        self.combined.save('{}/gan/{}.h5'.format(CHECKPOINTS_DIR, name))

    def load_model(self, name):
        """
        Loads model from CHECKPOINTS_DIR
        :param name: model id
        """
        self.discriminator = load_model('{}/discr/{}.h5'.format(CHECKPOINTS_DIR, name),
                                        custom_objects={'MinibatchDiscrimination': MinibatchDiscrimination,
                                                        'wasserstein_loss': self.wasserstein_loss})
        self.generator = load_model('{}/gen/{}.h5'.format(CHECKPOINTS_DIR, name),
                                    custom_objects={'GeneWiseNoise': GeneWiseNoise,
                                                    'wasserstein_loss': self.wasserstein_loss},
                                    compile=False)
        self.combined = load_model('{}/gan/{}.h5'.format(CHECKPOINTS_DIR, name),
                                   custom_objects={'GeneWiseNoise': GeneWiseNoise,
                                                   'MinibatchDiscrimination': MinibatchDiscrimination,
                                                   'wasserstein_loss': self.wasserstein_loss})
        self._latent_dim = self.generator.input_shape[-1]


def normalize(expr, kappa=1):
    """
    Normalizes expressions to make each gene have mean 0 and std kappa^-1
    :param expr: matrix of gene expressions. Shape=(nb_samples, nb_genes)
    :param kappa: kappa^-1 is the gene std
    :return: normalized expressions
    """
    mean = np.mean(expr, axis=0)
    std = np.std(expr, axis=0)
    return (expr - mean) / (kappa * std)


def restore_scale(expr, mean, std):
    """
    Makes each gene j have mean_j and std_j
    :param expr: matrix of gene expressions. Shape=(nb_samples, nb_genes)
    :param mean: vector of gene means. Shape=(nb_genes,)
    :param std: vector of gene stds. Shape=(nb_genes,)
    :return: Rescaled gene expressions
    """
    return expr * std + mean


def clip_outliers(expr, r_min, r_max):
    """
    Clips expression values to make them be between r_min and r_max
    :param expr: matrix of gene expressions. Shape=(nb_samples, nb_genes)
    :param r_min: minimum expression value (float)
    :param r_max: maximum expression value (float)
    :return: Clipped expression matrix
    """
    expr_c = np.copy(expr)
    expr_c[expr_c < r_min] = r_min
    expr_c[expr_c > r_max] = r_max
    return expr_c


def generate_data(gan, size, mean, std, r_min, r_max):
    """
    Generates size samples from generator
    :param gan: Keras GAN
    :param size: Number of samples to generate
    :param mean: vector of gene means. Shape=(nb_genes,)
    :param std: vector of gene stds. Shape=(nb_genes,)
    :param r_min: vector of minimum expression values for each gene
    :param r_max: vector of maximum expression values for each gene
    :return: matrix of expressions
    """
    expr = gan.generate_batch(size)
    expr = normalize(expr)
    expr = restore_scale(expr, mean, std)
    #expr = clip_outliers(expr, r_min, r_max)
    return expr

In [11]:
# Load data
expr, gene_symbols, sample_names = load_data()

file_name = 'MTB_overexpr_n{}_test30k'.format(len(gene_symbols))

Found 4026 genes in datafile


In [12]:
# Standardize data
data = normalize(expr, kappa=2)
#data = expr

In [13]:
# Train GAN
gan = GAN(data, gene_symbols, latent_dim=LATENT_DIM, noise_rate=DEFAULT_NOISE_RATE)
gan.train(epochs=30000, file_name=file_name, batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 4026)              0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 4026)              0         
_________________________________________________________________
minibatch_discrimination_2 ( (None, 4036)              201300    
_________________________________________________________________
dense_10 (Dense)             (None, 512)               2066944   
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
__________

101 [D loss: -7.6212] [G loss: 2.1735]
102 [D loss: -8.0666] [G loss: 1.7432]
103 [D loss: -7.2889] [G loss: 1.7543]
104 [D loss: -8.0834] [G loss: 2.0675]
105 [D loss: -7.4277] [G loss: 0.4813]
106 [D loss: -8.5973] [G loss: 2.6933]
107 [D loss: -8.0594] [G loss: 1.6320]
108 [D loss: -8.0119] [G loss: 1.6947]
109 [D loss: -8.1966] [G loss: 0.4169]
110 [D loss: -7.5985] [G loss: 1.5574]
111 [D loss: -8.2796] [G loss: 1.1886]
112 [D loss: -8.1939] [G loss: -0.0459]
113 [D loss: -7.3583] [G loss: 1.1556]
114 [D loss: -8.3318] [G loss: 1.3415]
115 [D loss: -8.1406] [G loss: 1.7444]
116 [D loss: -8.6090] [G loss: 2.2035]
117 [D loss: -8.6041] [G loss: 2.3448]
118 [D loss: -8.1304] [G loss: 2.9255]
119 [D loss: -8.2450] [G loss: 2.6973]
120 [D loss: -8.2685] [G loss: 3.3139]
121 [D loss: -7.7799] [G loss: 3.8169]
122 [D loss: -8.7141] [G loss: 3.8790]
123 [D loss: -9.1105] [G loss: 4.1344]
124 [D loss: -7.6919] [G loss: 4.2078]
125 [D loss: -8.0910] [G loss: 4.4106]
126 [D loss: -8.3347] [G

311 [D loss: -8.1020] [G loss: 3.1648]
312 [D loss: -8.5267] [G loss: 3.4937]
313 [D loss: -8.6502] [G loss: 3.0482]
314 [D loss: -8.1916] [G loss: 2.7099]
315 [D loss: -9.1354] [G loss: 3.2235]
316 [D loss: -7.7361] [G loss: 2.8914]
317 [D loss: -7.5389] [G loss: 3.3563]
318 [D loss: -9.2872] [G loss: 2.5777]
319 [D loss: -8.0470] [G loss: 2.3939]
320 [D loss: -7.9849] [G loss: 2.4764]
321 [D loss: -7.9127] [G loss: 2.6158]
322 [D loss: -8.7914] [G loss: 2.9290]
323 [D loss: -7.4244] [G loss: 2.5468]
324 [D loss: -8.1763] [G loss: 2.4677]
325 [D loss: -9.0450] [G loss: 2.2754]
326 [D loss: -8.6376] [G loss: 3.0843]
327 [D loss: -7.8267] [G loss: 1.8402]
328 [D loss: -10.0530] [G loss: 2.6844]
329 [D loss: -7.8498] [G loss: 2.9382]
330 [D loss: -8.9558] [G loss: 3.1825]
331 [D loss: -8.0842] [G loss: 2.4113]
332 [D loss: -8.9518] [G loss: 3.3730]
333 [D loss: -7.7253] [G loss: 2.9165]
334 [D loss: -9.0819] [G loss: 3.3079]
335 [D loss: -8.2647] [G loss: 2.9256]
336 [D loss: -8.4742] [G

520 [D loss: -8.3977] [G loss: -0.7268]
521 [D loss: -8.5604] [G loss: -0.6777]
522 [D loss: -7.6031] [G loss: 0.3323]
523 [D loss: -7.6850] [G loss: 0.1549]
524 [D loss: -8.2023] [G loss: 0.3741]
525 [D loss: -8.2844] [G loss: 1.4170]
526 [D loss: -8.0603] [G loss: 1.6308]
527 [D loss: -8.9336] [G loss: 2.4065]
528 [D loss: -8.2490] [G loss: 2.8012]
529 [D loss: -8.3383] [G loss: 3.7154]
530 [D loss: -9.5955] [G loss: 4.3258]
531 [D loss: -9.4650] [G loss: 4.2896]
532 [D loss: -8.9004] [G loss: 4.9970]
533 [D loss: -9.5545] [G loss: 5.2038]
534 [D loss: -9.5220] [G loss: 4.9895]
535 [D loss: -8.6927] [G loss: 5.1053]
536 [D loss: -8.7869] [G loss: 4.6083]
537 [D loss: -9.7208] [G loss: 4.6546]
538 [D loss: -9.2630] [G loss: 4.0861]
539 [D loss: -8.8266] [G loss: 3.1707]
540 [D loss: -7.9513] [G loss: 3.0227]
541 [D loss: -8.2709] [G loss: 2.0088]
542 [D loss: -8.8199] [G loss: 1.9097]
543 [D loss: -7.9860] [G loss: 2.0149]
544 [D loss: -8.5866] [G loss: 1.1344]
545 [D loss: -8.2761] [

728 [D loss: -9.8904] [G loss: 11.4618]
729 [D loss: -9.4128] [G loss: 10.7637]
730 [D loss: -9.7140] [G loss: 10.1529]
731 [D loss: -9.7098] [G loss: 10.2733]
732 [D loss: -8.2383] [G loss: 8.8330]
733 [D loss: -7.2177] [G loss: 5.8337]
734 [D loss: -6.5075] [G loss: 5.6540]
735 [D loss: -6.9682] [G loss: 3.3745]
736 [D loss: -6.6701] [G loss: 2.3401]
737 [D loss: -6.2008] [G loss: -1.5719]
738 [D loss: -3.7499] [G loss: -2.9202]
739 [D loss: -3.0977] [G loss: -6.1488]
740 [D loss: -3.0733] [G loss: -9.0236]
741 [D loss: -1.6820] [G loss: -10.3292]
742 [D loss: -1.1222] [G loss: -11.9495]
743 [D loss: -1.3631] [G loss: -9.2774]
744 [D loss: -1.5343] [G loss: -10.9481]
745 [D loss: -3.2360] [G loss: -7.9918]
746 [D loss: -3.8275] [G loss: -6.6247]
747 [D loss: -6.1720] [G loss: -4.2818]
748 [D loss: -5.8817] [G loss: -4.1139]
749 [D loss: -7.6862] [G loss: -1.0620]
750 [D loss: -7.8497] [G loss: 1.2782]
751 [D loss: -7.8204] [G loss: 2.7729]
752 [D loss: -8.9942] [G loss: 4.4974]
753 [

...

29249 [D loss: -49.3750] [G loss: 37.7033]
29250 [D loss: -36.6613] [G loss: -88.2334]
29251 [D loss: -37.8472] [G loss: -208.7769]
29252 [D loss: -233.7236] [G loss: 228.3723]
29253 [D loss: -197.9112] [G loss: 272.0711]
29254 [D loss: -251.8289] [G loss: -47.6982]
29255 [D loss: -59.7591] [G loss: -140.1788]
29256 [D loss: -195.5659] [G loss: -115.2915]
29257 [D loss: -21.0240] [G loss: -68.2352]
29258 [D loss: -36.5102] [G loss: -39.4620]
29259 [D loss: -11.3997] [G loss: -42.4056]
29260 [D loss: -77.5974] [G loss: -207.4000]
29261 [D loss: -143.6902] [G loss: -159.4180]
29262 [D loss: -188.6340] [G loss: -16.7405]
29263 [D loss: -89.3416] [G loss: -17.9698]
29264 [D loss: -238.6280] [G loss: -3.1554]
29265 [D loss: -52.5212] [G loss: 130.0654]
29266 [D loss: -310.6819] [G loss: 325.9039]
29267 [D loss: -346.7771] [G loss: 456.8250]
29268 [D loss: -77.5502] [G loss: 7.1938]
29269 [D loss: -141.1698] [G loss: 247.9646]
29270 [D loss: -102.2061] [G loss: 226.6467]
29271 [D loss: -35.7

29434 [D loss: -51.0415] [G loss: -139.1023]
29435 [D loss: -38.1165] [G loss: 124.3013]
29436 [D loss: -107.0659] [G loss: 81.8521]
29437 [D loss: -153.9470] [G loss: 81.8534]
29438 [D loss: -164.5893] [G loss: 232.1347]
29439 [D loss: -127.3790] [G loss: -40.6373]
29440 [D loss: -78.4441] [G loss: -12.8826]
29441 [D loss: -66.9418] [G loss: 166.2353]
29442 [D loss: -169.8578] [G loss: 107.4254]
29443 [D loss: -240.6205] [G loss: 174.8066]
29444 [D loss: -341.1776] [G loss: 263.5930]
29445 [D loss: -188.9802] [G loss: 327.9562]
29446 [D loss: -250.3852] [G loss: 263.4103]
29447 [D loss: -38.1770] [G loss: 47.4675]
29448 [D loss: -221.0600] [G loss: 291.7281]
29449 [D loss: -42.7545] [G loss: 43.2452]
29450 [D loss: -183.9965] [G loss: -107.8538]
29451 [D loss: -118.6940] [G loss: -71.7212]
29452 [D loss: -202.4732] [G loss: 49.4462]
29453 [D loss: -256.5250] [G loss: 115.5407]
29454 [D loss: -142.4742] [G loss: -127.3517]
29455 [D loss: -71.2201] [G loss: 120.6556]
29456 [D loss: -300

29619 [D loss: -145.9612] [G loss: 142.2000]
29620 [D loss: -273.3088] [G loss: -27.2549]
29621 [D loss: -64.5421] [G loss: 49.2945]
29622 [D loss: -179.1815] [G loss: -105.1634]
29623 [D loss: -71.4213] [G loss: 788.4536]
29624 [D loss: -100.2038] [G loss: 211.7636]
29625 [D loss: -14.7901] [G loss: 399.0471]
29626 [D loss: -40.2563] [G loss: -45.7864]
29627 [D loss: -87.0889] [G loss: 943.6397]
29628 [D loss: -203.5228] [G loss: 192.5133]
29629 [D loss: -179.2469] [G loss: 222.5573]
29630 [D loss: -129.8014] [G loss: 378.1935]
29631 [D loss: -67.5467] [G loss: 155.4721]
29632 [D loss: -18.9351] [G loss: -169.0476]
29633 [D loss: -67.8374] [G loss: -148.1113]
29634 [D loss: -20.3433] [G loss: 77.3712]
29635 [D loss: -143.4649] [G loss: 313.8245]
29636 [D loss: -8.3396] [G loss: 53.7578]
29637 [D loss: -116.3274] [G loss: 104.6233]
29638 [D loss: -112.6124] [G loss: -150.2300]
29639 [D loss: -89.7851] [G loss: -154.7319]
29640 [D loss: -152.5910] [G loss: -160.6112]
29641 [D loss: -144

29804 [D loss: -38.4890] [G loss: 145.3902]
29805 [D loss: -90.3695] [G loss: -75.5865]
29806 [D loss: -121.6951] [G loss: 50.5933]
29807 [D loss: -92.1171] [G loss: 9.7848]
29808 [D loss: -113.8606] [G loss: 222.2152]
29809 [D loss: -114.3997] [G loss: 47.5641]
29810 [D loss: -164.6187] [G loss: -68.1006]
29811 [D loss: -132.0030] [G loss: -46.6260]
29812 [D loss: -15.3376] [G loss: 467.4954]
29813 [D loss: -92.6634] [G loss: -53.4031]
29814 [D loss: -82.4401] [G loss: -126.6182]
29815 [D loss: -196.1387] [G loss: -15.9584]
29816 [D loss: -225.0274] [G loss: 1.9214]
29817 [D loss: -196.0664] [G loss: 61.3800]
29818 [D loss: -202.6294] [G loss: -26.0082]
29819 [D loss: -118.6609] [G loss: -14.8809]
29820 [D loss: -100.6632] [G loss: 46.4091]
29821 [D loss: -50.5996] [G loss: 195.2175]
29822 [D loss: -56.0334] [G loss: 330.9087]
29823 [D loss: -151.3196] [G loss: 120.8805]
29824 [D loss: -184.7509] [G loss: 370.1829]
29825 [D loss: -428.9199] [G loss: 247.6801]
29826 [D loss: -19.8893] 

29990 [D loss: -143.4154] [G loss: -217.0285]
29991 [D loss: -25.4131] [G loss: 16.2379]
29992 [D loss: -88.8291] [G loss: -86.9352]
29993 [D loss: -140.5506] [G loss: -65.1754]
29994 [D loss: -98.7398] [G loss: -21.3733]
29995 [D loss: -284.7738] [G loss: 119.7536]
29996 [D loss: -220.9435] [G loss: 182.8007]
29997 [D loss: -89.6198] [G loss: 322.6894]
29998 [D loss: -28.9334] [G loss: -238.3168]
29999 [D loss: -19.8529] [G loss: 71.8383]


In [8]:
# Generate synthetic data
mean = np.mean(expr, axis=0)
std = np.std(expr, axis=0)
r_min = expr.min()
r_max = expr.max()
s_expr = generate_data(gan, expr.shape[0], mean, std, r_min, r_max)

In [9]:
# Save generated data
save_synthetic(file_name, s_expr, gene_symbols)



In [10]:
#Save model
gan.save_model(file_name)
#NOTE: The model is saved while training now